In [142]:
from IPython.display import display, Math, Latex
from itertools import product, combinations
import copy
class monomial:
    #just initialization
    def __init__(self, fixed_part, running_indices):
        self.fixed_part = fixed_part.copy()
        self.running_indices = running_indices.copy()
        self.list_of_indices = {elem[0] : elem[1] for elem in running_indices.keys()}
    #gathering a latex style string to represent monomial    
    def __str__(self):
        result = ""
        for i in self.fixed_part.keys():
            result += "x_" + str(i)
            if self.fixed_part[i] > 1:
                result += "^{" + str(self.fixed_part[i]) + "}"
        for i in self.running_indices.keys():
            result += "x_{" + i[0] + "+" + str(i[1]) + "}"
            if self.running_indices[i] > 1:
                result += "^{" + str(self.running_indices[i]) + "}"
        result = result.replace("+0", "")
        if len(result) == 0:
            result = "1"
        return result
    
    #overriding addition
    def __add__(self, other):
        if type(self) == type(other):
            if self == other:
                return polynomial({self : 2})
            else:
                return polynomial({self : 1, other : 1})
        elif type(other) == polynomial:
            result = other.monomial_dict
            if self in result.monomial_dict.keys():
                result[self] += 1
            else:
                result[self] = 1
            return polynomial(result)
            
    #overriding subtraction
    def __sub__(self, other):
        if type(self) == type(other):
            if self == other:
                return polynomial({})
            else:
                return polynomial({self : 1, other : -1})
    #overriding multiplication
    def __mul__(self, other):
        if type(self) == type(other):
            result_fix = self.fixed_part.copy()
            result_run = self.running_indices.copy()
            for elem in other.fixed_part.keys():
                if elem in result_fix.keys():
                    result_fix[elem] += other.fixed_part[elem]
                else:
                    result_fix[elem] = other.fixed_part[elem]
            for elem in other.running_indices.keys():
                if elem in result_run.keys():
                    result_run[elem] += other.running_indices[elem]
                else:
                    result_run[elem] = other.running_indices[elem]
            return monomial(result_fix, result_run)
        elif type(other) == polynomial:
            result = polynomial({self * elem : other.monomial_dict[elem] for elem in other.monomial_dict.keys()})
            return result
    def __rmul__(self, other):
        return self.__mul__(other)
    #equality check
    def __eq__(self, other):
        return ((self.fixed_part == other.fixed_part) and (self.running_indices == other.running_indices))
    #makes hash out from string representation in order to use monomials as dict keys
    def __hash__(self):
        return hash(self.__str__())
    #changes index_to_change into what_to_insert(must be of form ("index", number added to "index"))
    #this method changes self
    def self_substitution(self, index_to_change, what_to_insert):
        if index_to_change in self.list_of_indices.keys():
            if what_to_insert[0] == "":
                helper = self.running_indices.copy()
                for elem in helper.keys():
                    if index_to_change == elem[0]:
                        b = what_to_insert[1] + elem[1]
                        deg = self.running_indices[elem]
                        self.running_indices.pop(elem)
                        if b in self.fixed_part.keys():
                            self.fixed_part[b] += deg
                        else:
                            self.fixed_part[b] = deg
            else:
                helper = self.running_indices.copy()
                for elem in helper.keys():
                    if index_to_change == elem[0]:
                        b = what_to_insert[1] + elem[1]
                        deg = self.running_indices[elem]
                        self.running_indices.pop(elem)
                        if (what_to_insert[0], b) in self.running_indices.keys():
                            self.running_indices[(what_to_insert[0], b)] += deg
                        else:
                            self.running_indices[(what_to_insert[0], b)] = deg
                #self.list_of_indices.pop(index_to_change)
                #self.list_of_indices[what_to_insert[0]] = b
                
    #does the same as self_substitution, without changing self, but creating a copy        
    def copy_substitution(self, index_to_change, what_to_insert):
        result = monomial(self.fixed_part, self.running_indices)
        result.self_substitution(index_to_change, what_to_insert)
        return result
    
    #if return_gcd is true returns a three element tuple (gcd, self freed from gcd, other freed from gcd)
    #otherwise returns only freed self and other
    def free_from_gcd(self, other, return_gcd):
        self_copy = monomial(self.fixed_part, self.running_indices)
        other_copy = monomial(other.fixed_part, other.running_indices)
        sure_fixed_part = {}
        sure_running_indices = {}
        helper = self_copy.running_indices.copy()
        for elem in helper.keys():
            if elem in other_copy.running_indices.keys():
                if self_copy.running_indices[elem] == other_copy.running_indices[elem]:
                    sure_running_indices[elem] = self_copy.running_indices[elem]
                    self_copy.running_indices.pop(elem)
                    other_copy.running_indices.pop(elem)
                elif other_copy.running_indices[elem] < self_copy.running_indices[elem]:
                    sure_running_indices[elem] = other_copy.running_indices[elem]
                    self_copy.running_indices[elem] -= other_copy.running_indices[elem]
                    other_copy.running_indices.pop(elem)
                else:
                    sure_running_indices[elem] = self_copy.running_indices[elem]
                    other_copy.running_indices[elem] -= self_copy.running_indices[elem]
                    self_copy.running_indices.pop(elem)
        helper = self_copy.fixed_part.copy()
        for elem in helper.keys():
            if elem in other_copy.fixed_part.keys():
                if self_copy.fixed_part[elem] == other_copy.fixed_part[elem]:
                    sure_fixed_part[elem] = self_copy.fixed_part[elem]
                    self_copy.fixed_part.pop(elem)
                    other_copy.fixed_part.pop(elem)
                elif other_copy.fixed_part[elem] < self_copy.fixed_part[elem]:
                    sure_fixed_part[elem] = other_copy.fixed_part[elem]
                    self_copy.fixed_part[elem] -= other_copy.fixed_part[elem]
                    other_copy.fixed_part.pop(elem)
                else:
                    sure_fixed_part[elem] = self_copy.fixed_part[elem]
                    other_copy.fixed_part[elem] -= self_copy.fixed_part[elem]
                    self_copy.running_indices.pop(elem)
        if return_gcd:
            return (monomial(sure_fixed_part, sure_running_indices), self_copy, other_copy)
        else:
            return (self_copy, other_copy)
    
    #finds any possible substitution leading to self and other having common variables
    def possible_intersections(self, other):
        possible_running_indices = {}
        self_copy, other_copy = self.free_from_gcd(other, return_gcd = False)
        #display(Math(self_copy.__str__()))
        #display(Math(other_copy.__str__()))
        for elem in self_copy.running_indices.keys():
            listik = []
            for elem1 in other_copy.fixed_part.keys():
                listik.append(("", elem1 - elem[1]))
                #print(elem[0], "should be equal to ", elem1 - elem[1])
            for elem2 in other_copy.running_indices.keys():
                if elem2[0] != elem[0]:
                    listik.append((elem2[0], elem2[1] - elem[1]))
                    #print(elem[0], "should be equal to ", elem2[0], "+", elem2[1] - elem[1])
            if len(listik) > 0:
                possible_running_indices[elem[0]] = listik
        for elem in other_copy.running_indices.keys():
            listik = []
            for elem1 in self_copy.fixed_part.keys():
                listik.append(("", elem1 - elem[1]))
            if elem[0] in possible_running_indices.keys():
                possible_running_indices[elem[0]].extend(listik)
            elif len(listik) > 0:
                possible_running_indices[elem[0]] = listik
        result = [list(set([(key, value)  for value in possible_running_indices[key]]))
                  for key in possible_running_indices.keys()]
        return result
    
    #just pretty latex printing function
    def print(self):
        display(Math(self.__str__()))
            

class polynomial:
    
    #just initialization
    def __init__(self, monomial_dict):
        #self.monomial_dict = monomial_dict.copy()
        self.monomial_dict = copy.deepcopy(monomial_dict)
        for elem in monomial_dict.keys():
            if self.monomial_dict[elem] == 0:
                self.monomial_dict.pop(elem)
                
    #overriding addition for polynomials            
    def __add__(self, other):
        if type(self) == type(other): 
            result_dict = copy.deepcopy(self.monomial_dict)
            for elem in other.monomial_dict.keys():
                if elem in result_dict.keys():
                    result_dict[elem] += other.monomial_dict[elem]
                else:
                    result_dict[elem] = other.monomial_dict[elem]
            return polynomial(result_dict)
        elif type(other) == monomial:
            return other * self
        
    #overriding subtraction for polynomials
    def __sub__(self, other):
        if type(self) == type(other):
            result_dict = copy.deepcopy(self.monomial_dict)
            for elem in other.monomial_dict.keys():
                if elem in result_dict.keys():
                    result_dict[elem] -= other.monomial_dict[elem]
                else:
                    result_dict[elem] = -other.monomial_dict[elem]
            return polynomial(result_dict)
    #overriding multiplication for polynomials
    def __mul__(self, other):
        if type(self) == type(other):
            result = polynomial({})
            for elem in self.monomial_dict.keys():
                helper = elem * other
                for elem1 in helper.monomial_dict.keys():
                    helper.monomial_dict[elem1] *= self.monomial_dict[elem]
                result += helper
            return result
        elif type(other) == monomial:
            return other * self
    #gathering a latex style string in order to represent a polynomial
    def __str__(self):
        result = ""
        for elem in self.monomial_dict.keys():
            if self.monomial_dict[elem] != 1 and self.monomial_dict[elem] != -1:
                result += "+" + str(self.monomial_dict[elem]) + elem.__str__()
            elif self.monomial_dict[elem] == -1:
                result += "-" + elem.__str__()
            else:
                result += "+" + elem.__str__()
        result = result.replace("+-", "-")
        result = result.replace("++", "+")
        if result[0] == "+":
            result = result[1 : ]
        if len(result) == 0:
            result = "0"
        return result
    #performs a substitution same to the self_substitution for monomials
    def self_substitution(self, index_to_change, what_to_insert):
        for elem in self.monomial_dict.keys():
            elem.substitution(index_to_change, what_to_insert)
    #performs a substitution same to the copy_substitution for monomials
    def copy_substitution(self, index_to_change, what_to_insert):
        result = {elem.copy_substitution(index_to_change, what_to_insert) : self.monomial_dict[elem] 
                  for elem in self.monomial_dict.keys()}
        return polynomial(result)
    #pretty latex printing function
    def print(self):
        display(Math(self.__str__()))
        
#def S_polynomial(polynomial_1, polynomial_2, leading_monomial_1, leading_monomial_2):
#    fix_leader_1 = {elem : leading_monomial_1.fixed_part[elem] if not (elem in leading_monomial_2.fixed_part.keys()) 
#                    for elem in leading_monomial_1.fixed_part.keys()}

In [129]:
m1 = monomial(fixed_part = {0 : 1, 5 : 2}, running_indices = {("k_1", 2) :  1, ("p", 5) : 4})

In [130]:
m2 = monomial({0 : 1, 7 : 1}, {("k_1", 1) : 9, ("l", 1) : 5})

In [107]:
p = m1.free_from_gcd(m2)

In [111]:
m1.print()
m2.print()
print("gcd is:")
p[0].print()
p[1].print()
p[2].print()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

gcd is:


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [131]:
m1.print()
m2.print()
dict5 = m1.possible_intersections(m2)
dict6 = m2.possible_intersections(m1)

print(dict5)
print(dict6)


<IPython.core.display.Math object>

<IPython.core.display.Math object>

[[('p', ('l', -4)), ('p', ('', 2)), ('p', ('k_1', -4))], [('k_1', ('', 4)), ('k_1', ('l', -1)), ('k_1', ('', 5))], [('l', ('', 4))]]
[[('p', ('', 2))], [('k_1', ('', 4)), ('k_1', ('p', 4)), ('k_1', ('', 5))], [('l', ('k_1', 1)), ('l', ('', 4)), ('l', ('p', 4))]]


In [132]:
p = list(product(*dict5))

In [141]:
list(combinations(p[0], 3))

[(('p', ('l', -4)), ('k_1', ('', 4)), ('l', ('', 4)))]

In [136]:
x

[(('p', ('l', -4)), ('k_1', ('', 4))),
 (('p', ('l', -4)), ('l', ('', 4))),
 (('k_1', ('', 4)), ('l', ('', 4)))]

In [143]:
m1 = monomial({0 : 1}, {("k", 1) : 1})
m2 = monomial({1 : 1}, {("k", 0) : 1})
m3 = monomial({0 : 1}, {("l", 1) : 1})
m4 = monomial({1 : 1}, {("l", 0) : 1})
f1 = m1 - m2
f2 = m3 - m4
f1.print()
f2.print()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [144]:
f3 = f1 - f2

In [145]:
f3.print()

<IPython.core.display.Math object>

In [96]:
f1 = m1 + m2
f2 = m1 * m2 - m1

In [97]:
f1.print()
f2.print()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [98]:
f3 = f1 * f2

In [99]:
f3.print()

<IPython.core.display.Math object>

In [86]:
m3 = m1 * m2

In [87]:
m3.print()

<IPython.core.display.Math object>

In [92]:
m1.print()
m2.print()
m3.print()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [89]:
m4 = m3 * m2

In [91]:
m4.print()
print(m4.fixed_part, m4.running_indices)

<IPython.core.display.Math object>

{0: 3, 5: 2, 7: 2} {('k_1', 2): 1, ('k_1', 1): 18, ('p', 5): 4, ('l', 1): 10}
